CLASE UTILIZADA PARA LA ECHO STATE NETWORK CON TORCH

In [ ]:
import torch
import torch.nn as nn

class ESN(torch.nn.Module):
    def __init__(self, input_dim, reservoir_size, output_dim, leaking_rate=0.3, spectral_radius=0.9):
        super().__init__()
        
        self.reservoir_size = reservoir_size
        self.leaking_rate = leaking_rate
        
        self.W_in = nn.Parameter(torch.randn(reservoir_size, input_dim) * 0.5, requires_grad=False)
        
        W = torch.randn(reservoir_size, reservoir_size)
        W *= spectral_radius / torch.max(torch.abs(torch.linalg.eigvals(W)))
        
        connectivity_mask = torch.rand(reservoir_size, reservoir_size) < 0.1
        self.W = nn.Parameter(W * connectivity_mask.float(), requires_grad=False)
        
        self.W_out = nn.Linear(reservoir_size, output_dim)
        
    def forward(self, inputs):
        batch_size, seq_len, input_dim = inputs.shape
        states = torch.zeros(batch_size, self.reservoir_size)
        
        all_states = []
        for t in range(seq_len):
            current_input = inputs[:,t,:]
            preactivation = torch.mm(current_input, self.W_in.T) + torch.mm(states, self.W.T)
            states = (1 - self.leaking_rate)*states + self.leaking_rate * torch.tanh(preactivation)
            all_states.append(states.unsqueeze(1))
            
        return torch.cat(all_states, dim=1)

In [ ]:
# Convertir datos de entrenamiento
X_train = torch.from_numpy(X_train).float()  # Shape (n, 100, 62)
y_train = torch.from_numpy(y_train).long()   # Shape (n, 1)

In [ ]:
esn = ESN(input_dim=62, reservoir_size=200, output_dim=3)
optimizer = torch.optim.Adam(esn.parameters(), lr=1e-3, weight_decay=1e-2)

# Entrenamiento
for epoch in range(15):
    print("Epoch ", epoch)
    reservoir_states = esn(X_train)
    outputs = esn.W_out(reservoir_states[:,-1,:])  # Último estado para clasificación
    
    loss = torch.nn.CrossEntropyLoss()(outputs, y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [ ]:
# Convertir datos de entrenamiento
X_test = torch.from_numpy(X_test).float()  # Shape (n, 100, 62)
y_test = torch.from_numpy(y_test).long()   # Shape (n, 1)

In [ ]:
# Evaluación
with torch.no_grad():
    test_states = esn(X_test)
    y_pred = torch.argmax(esn.W_out(test_states[:,-1,:]), dim=1)
    accuracy = (y_pred == y_test).float().mean()

In [ ]:
print("RESULTADOS DE APROXIMADAMENTE EL 33% DE ACIERTO")